In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 16:59:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [10]:
df_green.registerTempTable('green')

/home/polina/spark/spark-3.4.4-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
df_green_revenue = spark.sql("""
SELECT 
    -- Revenue grouping 
    date_trunc("hour", lpep_pickup_datetime) AS hour,
    PULocationID AS zone,


    SUM(total_amount) AS total_amount,
    COUNT(1) AS count_records
FROM 
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
    AND
    lpep_pickup_datetime < '2022-01-01 00:00:00'
GROUP BY 
    1,2
;
""")

In [22]:
df_green_revenue.show()

+-------------------+----+------------------+-------------+
|               hour|zone|      total_amount|count_records|
+-------------------+----+------------------+-------------+
|2020-01-31 11:00:00| 220|            168.08|            6|
|2020-01-04 16:00:00| 196|210.44000000000005|           10|
|2020-01-04 20:00:00|  25|369.57000000000005|           23|
|2020-01-29 14:00:00| 203|             43.26|            2|
|2020-01-02 13:00:00|  74|1037.6299999999994|           71|
|2020-01-23 22:00:00|  35|             36.11|            2|
|2020-01-26 12:00:00|  35|            257.03|            8|
|2020-01-22 10:00:00|  74|1179.0599999999988|           83|
|2020-01-10 22:00:00| 216|            181.89|            5|
|2020-01-24 10:00:00| 260|            149.63|           10|
|2020-01-24 05:00:00| 133|             19.78|            1|
|2020-01-03 13:00:00| 250|              27.6|            2|
|2020-01-16 01:00:00|   7|             110.1|            9|
|2020-01-02 09:00:00|  66|             2

In [25]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [26]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [29]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Revenue grouping 
    date_trunc("hour", tpep_pickup_datetime) AS hour,
    PULocationID AS zone,


    SUM(total_amount) AS total_amount,
    COUNT(1) AS count_records
FROM 
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
    AND
    tpep_pickup_datetime < '2022-01-01 00:00:00'
GROUP BY 
    1,2
;
""")

In [30]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [31]:
df_yellow_revenue.show()

+-------------------+----+------------------+-------------+
|               hour|zone|      total_amount|count_records|
+-------------------+----+------------------+-------------+
|2020-01-29 18:00:00| 229| 5427.470000000005|          304|
|2020-01-24 15:00:00| 264| 2249.289999999999|           93|
|2020-01-21 18:00:00| 162|14090.389999999996|          816|
|2020-01-18 00:00:00| 144| 2951.300000000001|          174|
|2020-01-17 03:00:00| 132|2031.0099999999995|           46|
|2020-01-23 19:00:00| 224|327.28000000000003|           16|
|2020-01-06 08:00:00|  43| 2765.499999999998|          169|
|2020-01-04 11:00:00| 140| 1662.959999999999|          112|
|2020-01-04 23:00:00|  87| 744.2199999999999|           36|
|2020-01-26 17:00:00| 107| 4251.760000000004|          274|
|2020-01-31 13:00:00| 140| 4380.170000000002|          277|
|2020-01-31 21:00:00|  50|1784.4399999999994|          104|
|2020-01-08 08:00:00| 249| 2433.119999999999|          165|
|2020-01-17 08:00:00|  13|2948.720000000